##Twitter 단어 분석 APP입니다. 

**by Sun-il Kim**
*2019-01-10*

[github MachineLearningSamples](https://github.com/apostkor/MachineLearningSamples)

> 주석은 상단에 표시

**필요 모듈**
- tweepy -> pip install tweepy
- TextBlob -> pip install textblob 
 - 추가로 python -m textblob.download_corpora로 분석 데이터도 받아와야 합니다. 
 

In [ ]:

# 트위터 API의 커스터머키와 엑세스토큰을 인터넷상에 공개하는 것을 막기 위해 
# 환경변수에 키를 추가하고 os 모듈로 불러와 사용합니다. 
import config
import tweepy 
from textblob import TextBlob


# 커스터머키
consumer_key = config.api_key
consumer_secret = config.api_secret
# 엑세스토큰
access_token = config.access_token
access_token_secret = config.token_secret

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

q = input("분석할 검색어를 입력하세요 > ")
public_tweets = api.search('q',lang='en',count='100')

sum = 0

positive_count = 0
neutral_count = 0
negative_count = 0

for tweet in public_tweets:
    
    #print(tweet.text)
    analysis = TextBlob(tweet.text)
    #print(analysis.sentiment.polarity)
   
    if analysis.sentiment.polarity > 0.0:
        positive_count = positive_count + 1
        sample_pos = analysis
    elif analysis.sentiment.polarity == 0.0:
        neutral_count = neutral_count+1
    else:
        negative_count = negative_count + 1
        sample_neg = analysis
   


print ("ㄴ 상기 검색어 \"{}\"에 대해서 분석을 실시합니다!\n".format(q))

print ("---------------------여론 분석----------------------")
print ("약 {}% 의 트윗이 긍정적입니다!".format(positive_count))
print ("약 {}% 의 트윗이 중립적입니다!".format(neutral_count))
print ("약 {}% 의 트윗이 부정적입니다!\n".format(negative_count))

print ("---------------------트윗 예시----------------------")
print ("긍정적 트윗 예시:", sample_pos)
print ("부정적 트윗 예시:", sample_neg)